In [169]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import pandas as pd
import numpy as np
from IPython.display import clear_output

In [170]:
from ff_espn_api import League

In [185]:
mv_or_vl = 'mv'

In [186]:
if mv_or_vl == 'vl':
    # Varberg-Lindquist League
    league_id = 1086928
    year = 2019
    swid = input();
    espn_s2 = input();
    clear_output()
    league = League(league_id, year, espn_s2, swid)
else:
    # Mounds View League
    league_id = 200140
    year = 2019
    swid = input();
    espn_s2 = input();
    clear_output()
    league = League(league_id, year, espn_s2, swid)  

In [187]:
#### Preseason
# Replacement points dict for all positions
repl_points = {'QB': 263,
              'WR': 100,
              'TE': 83,
              'K': 130,
              'RB': 82,
              'D/ST': 93}
# Load FFA preseason projections
data_name = 'FFA_Projections_2019.csv'
pre_projections = pd.read_csv(data_name)
pre_projections = pre_projections.set_index('name_espn')
# give each player on roster preseason projections
for team in league.teams:
    for player in team.roster:
        if player.name in pre_projections.index:
            player.pre_proj_ppg = np.round(pre_projections.loc[player.name, 'Points']/16, 2)
        elif player.name == 'Mike Evans':
            player.pre_proj_ppg = np.round(186/16, 2)
        else:
            player.pre_proj_ppg = np.round(repl_points[player.position]/16, 2)
            print(player.name + ' not found so I guess I\'ll give him ' + str(player.pre_proj_ppg))

Jack Doyle not found so I guess I'll give him 5.19
T.J. Yeldon not found so I guess I'll give him 5.12
Jalen Richard not found so I guess I'll give him 5.12
Patriots D/ST not found so I guess I'll give him 5.81
Mason Crosby not found so I guess I'll give him 8.12
Kenny Stills not found so I guess I'll give him 6.25
Adam Humphries not found so I guess I'll give him 6.25
Mike Badgley not found so I guess I'll give him 8.12
Broncos D/ST not found so I guess I'll give him 5.81
Albert Wilson not found so I guess I'll give him 6.25
DaeSean Hamilton not found so I guess I'll give him 6.25
Jake Elliott not found so I guess I'll give him 8.12
Chase Edmonds not found so I guess I'll give him 5.12
Mohamed Sanu not found so I guess I'll give him 6.25
Adam Vinatieri not found so I guess I'll give him 8.12
Deebo Samuel not found so I guess I'll give him 6.25
Eagles D/ST not found so I guess I'll give him 5.81
Trey Quinn not found so I guess I'll give him 6.25
Chiefs D/ST not found so I guess I'll gi

In [188]:
# Create Team best lineup pre_proj_ppg
if mv_or_vl == 'vl':
    slots = ['QB', 'QB', 'RB', 'RB', 'WR', 'WR', 'WR', 'TE', 'RB/WR/TE', 'K']
    std_dev = 25
    half_ppr_factor = 1
else:
    slots = ['QB', 'RB', 'RB', 'WR', 'WR', 'TE', 'RB/WR/TE', 'D/ST', 'K']
    std_dev = 25
    half_ppr_factor = 1.2
for team in league.teams:
    team.pre_proj_std_dev = std_dev
    team.pre_proj_ppg = 0
    players_used = []
    while len(players_used) < len(slots):
        for slot in slots:
            max_points = 0
            for player in team.roster:
                if (slot in player.eligibleSlots) & (player not in players_used) & (player.pre_proj_ppg > max_points):
                    max_points = player.pre_proj_ppg
                    max_player = player
            team.pre_proj_ppg += max_points
            players_used.append(max_player)
ave_pre_proj_ppg = sum(team.pre_proj_ppg for team in league.teams)/len(league.teams)
for team in league.teams:
    team.pre_proj_ppg_mod = np.round((team.pre_proj_ppg + ave_pre_proj_ppg)/2*half_ppr_factor,1)

In [189]:
for team in league.teams:
    print([team.team_name, team.pre_proj_ppg_mod, team.pre_proj_std_dev])

['Mob Badison', 111.0, 25]
['Ground’n Pound', 111.9, 25]
['Loading ...', 115.3, 25]
['Barberque Kittle Chips', 116.9, 25]
['12 Years A Freeman', 112.7, 25]
['Take Mahomes, Country Road ', 113.6, 25]
['United States  Of Kamaraca', 113.5, 25]
['Windy City Miracle', 121.4, 25]
['Saved By  Le’ Bell', 119.0, 25]
['Cheaper by  the Cousins 2', 116.3, 25]
['Duke in my Pants Johnson', 113.6, 25]
['Hot Chubb Time Machine', 118.0, 25]


In [190]:
#### Midseason
WEEK = league.teams[0].wins + league.teams[0].losses

if mv_or_vl == 'vl':
    # change this later so we don't need week
    NUM_WEEKS = 11
    NUM_PLAY_WEEKS = 6
else:
    NUM_WEEKS = 12
    NUM_PLAY_WEEKS = 4

In [191]:
# create schedule: list of weeks, each week is list of matchups
schedule = []
for game in range(WEEK,NUM_WEEKS):
    week_sched = []
    for team in league.teams: 
        if((team not in week_sched) & (team.schedule[game] not in week_sched)):
            week_sched.append(team) 
            week_sched.append(team.schedule[game])
    schedule.append(week_sched)

In [192]:
# lists for final dataframe
ppg_list = []
proj_ppg_list = []
if WEEK > 0:
    # change the mid season scoring
    for team in league.teams:
        ppg = np.round(sum(team.scores[0:WEEK])/WEEK,2)
        team.ppg = ppg
        team.proj_ppg = ppg
        std_dev = round(np.std(team.scores[0:WEEK]),1)
        team.std_dev = std_dev
        team.proj_std_dev = std_dev
        ppg_list.append(team.ppg)
        proj_ppg_list.append(team.proj_ppg)
else:
    for team in league.teams:
        team.proj_ppg = team.pre_proj_ppg_mod
        team.proj_std_dev = team.pre_proj_std_dev
        proj_ppg_list.append(team.proj_ppg)

In [194]:
for team in league.teams:
    team.sim_fst   = 0
    team.sim_semi   = 0
    team.sim_final  = 0
    team.sim_champ  = 0
#### Start iterations
## Sim Regular Season
iters = 30000
for iterations in range(0, iters):
    for team in league.teams:
        team.sim_wins       = team.wins
        team.sim_losses     = team.losses
        team.sim_weeks      = np.random.normal(team.proj_ppg, team.proj_std_dev, NUM_WEEKS-WEEK)
        team.sim_points_for = sum(team.sim_weeks)
    for week in range(0,len(schedule)):
        for game_num in range(0,len(schedule[week]),2):
            team1 = schedule[week][game_num]
            team2 = schedule[week][game_num+1]
            if(team1.sim_weeks[week] > team2.sim_weeks[week]):
                team1.sim_wins   += 1
                team2.sim_losses += 1
            else:
                team1.sim_losses += 1
                team2.sim_wins   += 1
    # Sort by sim_points_for, then sort by sim_wins to get standing
    standings = sorted(league.teams, key=lambda x: x.sim_points_for, reverse=True)
    standings = sorted(standings, key=lambda x: x.sim_wins, reverse=True)
    ## Sim Playoffs
    # Varberg-Lindquist
    if mv_or_vl == 'vl':
        for team in standings[0:7]:
            team.sim_fst +=1
        for team in standings[0:7]:
            team.sim_play = np.random.normal(2*team.proj_ppg, 2**(1/2)*team.proj_std_dev, int(NUM_PLAY_WEEKS/2))
        fst_rd = 0 # first element on sim_play
        semi_1 = standings[0]
        if standings[2].sim_play[fst_rd] > standings[5].sim_play[fst_rd]:
            semi_2 = standings[2]
        else:
            semi_2 = standings[5]
        if standings[3].sim_play[fst_rd] > standings[4].sim_play[fst_rd]:
            semi_3 = standings[3]
        else:
            semi_3 = standings[4]
        if standings[1].sim_play[fst_rd] > standings[6].sim_play[fst_rd]:
            semi_4 = standings[1]
        else:
            semi_4 = standings[6]
        semi_1.sim_semi += 1
        semi_2.sim_semi += 1
        semi_3.sim_semi += 1
        semi_4.sim_semi += 1
        semi = 1 # 2nd element of sim_play
        if semi_1.sim_play[semi] > semi_2.sim_play[semi]:
            final_1 = semi_1
        else:
            final_1 = semi_2
        if semi_3.sim_play[semi] > semi_4.sim_play[semi]:
            final_2 = semi_3
        else:
            final_2 = semi_4
        final_1.sim_final += 1
        final_2.sim_final += 1
        champ = 2 # 3rd element of sim_play
        if final_1.sim_play[champ] > final_2.sim_play[champ]:
            champ = final_1
        else:
            champ = final_2
        champ.sim_champ += 1
    # Mounds View
    else:
        for team in standings[0:6]:
            team.sim_fst += 1
        for team in standings[0:6]:
            team.sim_play = np.random.normal(team.proj_ppg,team.proj_std_dev, NUM_PLAY_WEEKS)
        fst_rd = 0
        semi_1 = standings[0]
        semi_4 = standings[1]
        if standings[2].sim_play[fst_rd] > standings[5].sim_play[fst_rd]:
            semi_2 = standings[2]
        else:
            semi_2 = standings[5]
        if standings[3].sim_play[fst_rd] > standings[4].sim_play[fst_rd]:
            semi_3 = standings[3]
        else:
            semi_3 = standings[4]
        semi_1.sim_semi += 1
        semi_2.sim_semi += 1
        semi_3.sim_semi += 1
        semi_4.sim_semi += 1
        semi = 1 # 2nd element of sim_play
        if semi_1.sim_play[semi] > semi_2.sim_play[semi]:
            final_1 = semi_1
        else:
            final_1 = semi_2
        if semi_3.sim_play[semi] > semi_4.sim_play[semi]:
            final_2 = semi_3
        else:
            final_2 = semi_4
        final_1.sim_final += 1
        final_2.sim_final += 1
        champ = [2,3] # 3rd and 4th element of sim_play
        if sum(final_1.sim_play[champ]) > sum(final_2.sim_play[champ]):
            champ = final_1
        else:
            champ = final_2
        champ.sim_champ += 1

In [195]:
# Create dataframe with teams data
tm_ids = []
tm_nm_list = []
abbrev = []
wins = []
losses = []
fst_list = []
semi_list = []
final_list = []
champ_list = []
for team in league.teams:
    tm_ids.append(team.team_id)
    tm_nm_list.append(team.team_name)
    abbrev.append(team.team_abbrev)
    wins.append(team.wins)
    losses.append(team.losses)
    fst_list.append(np.round(team.sim_fst/iters, 3))
    semi_list.append(np.round(team.sim_semi/iters, 3))
    final_list.append(np.round(team.sim_final/iters, 3))
    champ_list.append(np.round(team.sim_champ/iters, 3))
teamsdf = pd.DataFrame({'ID': tm_ids})
teamsdf = teamsdf.set_index('ID')
teamsdf['Owner'] = abbrev
teamsdf['Team'] = tm_nm_list
teamsdf['Wins'] = wins
teamsdf['Losses'] = losses
teamsdf['proj_ppg'] = proj_ppg_list
teamsdf['Playoffs'] = ["{:.0%}".format(x) for x in fst_list]
teamsdf['Semifinals'] = ["{:.0%}".format(x) for x in semi_list]
teamsdf['Finals'] = ["{:.0%}".format(x) for x in final_list]
teamsdf['Champion'] = champ_list
teamsdf = teamsdf.sort_values(by='Champion', ascending=False)
teamsdf['Champion'] = ["{:.0%}".format(x) for x in teamsdf['Champion']]
teamsdf

,Owner,Team,Wins,Losses,proj_ppg,Playoffs,Semifinals,Finals,Champion
ID,,,,,,,,,
8,VARB,Windy City Miracle,0,0,121.4,74%,57%,33%,19%
9,GRIF,Saved By Le’ Bell,0,0,119.0,65%,48%,26%,14%
12,HCTM,Hot Chubb Time Machine,0,0,118.0,62%,44%,23%,12%
4,KITT,Barberque Kittle Chips,0,0,116.9,56%,38%,20%,10%
10,WOLT,Cheaper by the Cousins 2,0,0,116.3,54%,36%,18%,9%
3,MORR,Loading ...,0,0,115.3,50%,33%,16%,8%
7,GRAN,United States Of Kamaraca,0,0,113.5,43%,26%,12%,6%
6,PLAZ,"Take Mahomes, Country Road",0,0,113.6,42%,26%,12%,5%
11,POOP,Duke in my Pants Johnson,0,0,113.6,43%,26%,12%,5%


In [196]:
if mv_or_vl == 'vl':
    path = '/Users/nickvarberg/Desktop/Life/Fantasy-Football/VL_Playoffs_Shiny/'
    name = 'VL_predictions_week_'+str(WEEK)+'.csv'
    pd.DataFrame.to_csv(teamsdf,path+name)
else:
    path = '/Users/nickvarberg/Desktop/Life/Fantasy-Football/MV_Playoffs_Shiny/'
    name = 'MV_predictions_week_'+str(WEEK)+'.csv'
    pd.DataFrame.to_csv(teamsdf,path+name)

In [230]:
# Start VL history df

# Start MV history df
historydf = teamsdf.loc[:,['Owner','Team']]
for col in ['proj_ppg','Playoffs','Semifinals','Finals','Champion']:
    name = col+'_'+str(WEEK)
    historydf[name] = teamsdf[col]
name = 'MV_predictions_history'
pd.DataFrame.to_csv(historydf,path+name)

In [231]:
historydf

,Owner,Team,proj_ppg_0,Playoffs_0,Semifinals_0,Finals_0,Champion_0
ID,,,,,,,
8,VARB,Windy City Miracle,121.4,74%,57%,33%,19%
9,GRIF,Saved By Le’ Bell,119.0,65%,48%,26%,14%
12,HCTM,Hot Chubb Time Machine,118.0,62%,44%,23%,12%
4,KITT,Barberque Kittle Chips,116.9,56%,38%,20%,10%
10,WOLT,Cheaper by the Cousins 2,116.3,54%,36%,18%,9%
3,MORR,Loading ...,115.3,50%,33%,16%,8%
7,GRAN,United States Of Kamaraca,113.5,43%,26%,12%,6%
6,PLAZ,"Take Mahomes, Country Road",113.6,42%,26%,12%,5%
11,POOP,Duke in my Pants Johnson,113.6,43%,26%,12%,5%
